This notebook analysis images with labeled image regions and shows how those labels can be transformed into a format that allows to train machine learning models for image segmentation.

In [ ]:
import rasterio
import matplotlib 
import numpy as np
import matplotlib.pyplot as plt
from rasterio.plot import show # import the show function which allows us to display the image
import geopandas as gpd

### Load and display metadata

In [ ]:
raster_file = '../../data/external/spacenet/3band_AOI_1_RIO_img5792.tif'
geojson_file = '../../data/external/spacenet/Geo_AOI_1_RIO_img5792.geojson'

In [ ]:
import rioxarray

In [ ]:
img_dataarr = rioxarray.open_rasterio(raster_file)

In [ ]:
img_dataarr

In [ ]:
img_dataarr.rio.crs

In [ ]:
img_dataarr.plot.imshow()
plt.show()

### Plot as image

In [ ]:
img_data = img_dataarr.values
img_data = np.transpose(img_data, (1, 2, 0))

In [ ]:
#img = np.dstack((red_band, green_band, blue_band))
f = plt.figure()
plt.imshow(img_data)
plt.show()

In [ ]:
building_footprints = gpd.read_file(geojson_file)
building_footprints.head(3)

In [ ]:
building_footprints.plot()
plt.show()

In [ ]:
fig, ax0 = plt.subplots(1, 1, figsize=(8,8))

img_dataarr.plot.imshow(ax=ax0)
building_footprints.plot(ax=ax0, alpha=0.25, edgecolor='None', facecolor='orange')
building_footprints.plot(ax=ax0, alpha=0.75, edgecolor='red', facecolor='None')
plt.show()

## Rasterize vector data

Burn vector data into raster file:

In [ ]:
from rasterio import features

In [ ]:
img_dataarr.rio.transform()

In [ ]:
# Rasterize vector using the shape and coordinate system of the raster
rasterized = features.rasterize(building_footprints.geometry.values,
                                out_shape = img_dataarr.rio.shape,
                                fill = 0,
                                out = None,
                                transform = img_dataarr.rio.transform(),
                                all_touched = False,
                                default_value = 1,
                                dtype = None)

In [ ]:
type(rasterized)

In [ ]:
# Plot raster
fig, (ax0, ax1) = plt.subplots(1, 2, figsize = (10, 10))

img_dataarr.plot.imshow(ax=ax0)
building_footprints.plot(ax=ax0, alpha=0.25, edgecolor='None', facecolor='orange')
building_footprints.plot(ax=ax0, alpha=0.75, edgecolor='red', facecolor='None')

show(rasterized, ax=ax1)
plt.show()

## Verify x/y indexing

The raster matrix is transposed in a sense that index with label `x` is stored in columns. This can be seen from the shape of the index:

In [ ]:
img_dataarr.indexes.dims

In [ ]:
img_dataarr.x.coords

In [ ]:
img_dataarr.shape

In [ ]:
img_data.shape

In [ ]:
rasterized.shape

In other words, the first dimension is `y`, while the second dimension is `x`. This is different for shapely geometries:

In [ ]:
this_poly = building_footprints.geometry[0]

In [ ]:
this_poly.exterior.coords.xy

In [ ]:
irow = 283 # rows, represents y coordinates
icol = 113 # columns, represents x coordinates

# usually in a tuple (irow, icol) the first index, irow, reflects rows. The second index, icol, reflects columns
# this convention is maintained for images as well, and the horizontal axis is called x, and the vertical axis y.
# Hence, (irow, icol) is a plotted as point (y, x) 

In [ ]:
img_data = img_dataarr.values.copy()
img_data = np.transpose(img_data, (1, 2, 0))

# make rectangular shape with color
this_fake_val = 250
img_data[0:10, 0:50, 0] = this_fake_val
img_data[0:10, 0:50, 1] = this_fake_val
img_data[0:10, 0:50, 2] = this_fake_val

In [ ]:
from matplotlib.patches import Circle

In [ ]:
img_data.shape

In [ ]:
this_circle = Circle((icol, irow), radius=5, color='red') # Circle coordinates are given as (x,y); irow represents y, icol represents x

In [ ]:
fig, ax = plt.subplots(1)
ax.imshow(img_data)
ax.add_patch(this_circle)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
(x_coord, y_coord) = rasterio.transform.xy(img_dataarr.rio.transform(), 0, 0)
x_coord, y_coord

In [ ]:
fig, ax = plt.subplots(1)
img_dataarr.plot.imshow(ax=ax)
ax.add_patch(Circle((x_coord, y_coord), radius=0.0001, color='red'))
plt.show()

In [ ]:
x_coord, y_coord = img_dataarr.rio.transform() * (icol, irow)

In [ ]:
x_coord, y_coord

In [ ]:
# (y_coord, x_coord) = rasterio.transform.xy(img_dataarr.rio.transform(), irow, icol)

In [ ]:
fig, ax = plt.subplots(1)
img_dataarr.plot.imshow(ax=ax)
ax.add_patch(Circle((x_coord, y_coord), radius=0.00002, color='red'))
plt.show()

## Measure distances

In [ ]:
from shapely import Point

In [ ]:
this_poly = building_footprints.geometry[0]
this_poly_gdp = gpd.GeoDataFrame(geometry=[this_poly], crs=img_dataarr.rio.crs)

fig, ax = plt.subplots(1)
img_dataarr.plot.imshow(ax=ax)

ax.add_patch(Circle((x_coord, y_coord), radius=0.00002, color='red'))
this_poly_gdp.plot(ax=ax, facecolor='None', edgecolor='blue')
plt.show()

In [ ]:
this_point = Point(x_coord, y_coord)
#this_point = Point(y_coord, x_coord)
#this_polygon = shape(feature['geometry'])
newpd = this_point.distance(this_poly.boundary)
newpd

In [ ]:
this_poly.bounds

In [ ]:
this_point.coords[0]

In [ ]:
counter = 0
min_counter = 0
running_min_dist = 100000
poly_contain_index = None

for this_poly in building_footprints.geometry:
    
    this_dist = this_point.distance(this_poly.boundary)
    if this_dist < running_min_dist:
        running_min_dist = this_dist
        min_counter = counter
    
    if this_poly.contains(this_point):
        poly_contain_index = counter
        
    counter += 1

In [ ]:
min_counter

In [ ]:
poly_contain_index

In [ ]:
this_poly = building_footprints.geometry[poly_contain_index]
this_poly_gdp = gpd.GeoDataFrame(geometry=[this_poly], crs=img_dataarr.rio.crs)

fig, ax = plt.subplots(1)
img_dataarr.plot.imshow(ax=ax)

ax.add_patch(Circle((x_coord, y_coord), radius=0.00002, color='red'))
this_poly_gdp.plot(ax=ax, facecolor='None', edgecolor='blue')
plt.show()

Compute minimum distance for each pixel

In [ ]:
this_point = Point(x_coord, y_coord)

In [ ]:
def get_min_distance(this_point, building_footprints):
    
    counter = 0
    running_min_dist = 100000
    
    for this_poly in building_footprints.geometry:
        
        this_dist = this_point.distance(this_poly.boundary)
        
        if this_poly.contains(this_point):
            
            this_dist = -1 * this_dist
            running_min_dist = this_dist
            
            return running_min_dist
        
        else:
            
            if this_dist < running_min_dist:
            
                running_min_dist = this_dist

    return running_min_dist

In [ ]:
n_rows, n_cols, _ = img_data.shape

In [ ]:
%%time

min_dist_matr = np.zeros((n_rows, n_cols))

for icol in range(0, n_cols):
    for irow in range(0, n_rows):
        x_coord, y_coord = img_dataarr.rio.transform() * (icol, irow)
        this_point = Point(x_coord, y_coord)
        
        min_dist = get_min_distance(this_point, building_footprints)
        min_dist_matr[irow, icol] = min_dist
        

In [ ]:
fig, (ax0, ax1) = plt.subplots(1, 2, figsize = (10, 10))

ax0.imshow(min_dist_matr, vmin=-0.00001, vmax=0.0001, cmap='jet_r')
#ax0.colorbar()

show(rasterized, ax=ax1)
plt.show()

In [ ]:
from scipy.ndimage import distance_transform_edt

In [ ]:
%%time

im_dist = distance_transform_edt(rasterized)

rasterized_inv = 1 - rasterized
im_dist_inv = distance_transform_edt(rasterized_inv)

Merge distance transforms

In [ ]:
im_dist_merged = (-1)*im_dist.copy()
im_dist_merged[im_dist_inv > 0] = im_dist_inv[im_dist_inv > 0]

In [ ]:
np.min(im_dist_merged)

In [ ]:
np.max(im_dist_merged)

In [ ]:
np.min(min_dist_matr), np.max(min_dist_matr)

In [ ]:
plt.imshow(im_dist_merged, vmin=-7, vmax=10, cmap='jet_r')
plt.colorbar()
plt.show()

In [ ]:
fig, (ax0, ax1) = plt.subplots(1, 2, figsize = (10, 10))

ax0.imshow(min_dist_matr, vmin=-0.00001, vmax=0.0001, cmap='jet_r')
#ax0.colorbar()

plt.imshow(im_dist_merged, vmin=-7, vmax=10, cmap='jet_r')
plt.show()

## Post-processing

In [ ]:
pos_values = (min_dist_matr > 0)*(1)
plt.imshow(pos_values)
plt.show()

In [ ]:
neg_values = (min_dist_matr < 0)*(-1)
plt.imshow(neg_values)
plt.show()

In [ ]:
building_estimates = (-1)*im_dist_merged.copy()
building_estimates[building_estimates < 0] = 0

In [ ]:
to_be_classified = building_estimates > 0 # all positive values are inside of a building
cluster_int = 1
all_clusters = building_estimates * 0

In [ ]:
plt.imshow(to_be_classified)

In [ ]:
def get_remaining_maximum(building_estimates, to_be_classified):
    """
    Find maximum value of remaining pixels. Positive values denote building interiors.
    """
    
    peak = np.argmax(np.multiply(building_estimates, to_be_classified))
    
    i_row = int(peak/building_estimates.shape[1])
    i_col = int(peak - i_row * building_estimates.shape[1])
    
    return i_row, i_col

def decreasing_neighbor_check(shift_matr, matr):
    """
    For a left-, right-, top- or bottom-shifted matrix, check whether new cluster neighbors are of decreasing slope
    """

    slope_matr = shift_matr - matr
    decreasing_neighbors = np.logical_and((slope_matr > 0), shift_matr > 0)#.astype(int)

    return decreasing_neighbors

In [ ]:
# building_estimates[i_row-2:i_row+2, i_col-2:i_col+2]

In [ ]:
# np.max(np.multiply(building_estimates, to_be_classified))

In [ ]:
# to_be_classified[i_row-2:i_row+2, i_col-2:i_col+2]

In [ ]:
while np.any(to_be_classified):
    
    if cluster_int > 100:
        print('Emergency break')
        break
        
    i_row, i_col = get_remaining_maximum(building_estimates, to_be_classified)
    
    # initialize current cluster
    this_cluster = building_estimates * 0
    this_cluster[i_row, i_col] = 1
    to_be_classified[i_row, i_col] = False

    cluster_not_final = True
    
    # start of WHILE loop

    while cluster_not_final:

        cluster_size = np.sum(this_cluster > 0)
        #print(f'Current cluster size: {cluster_size}')

        # grow current cluster with neighboring pixels (that are still to be classified) of decreasing values
        matr = this_cluster.copy()

        # create shifted matrices
        n_rows, n_cols = matr.shape
        col_zeros = np.zeros((n_rows, 1))
        row_zeros = np.zeros((1, n_cols))

        right_shift = np.hstack((col_zeros, matr[:, 0:-1]))
        left_shift = np.hstack((matr[:, 1:], col_zeros))
        top_shift = np.vstack((row_zeros, matr[0:-1, :]))
        down_shift = np.vstack((matr[1:, :], row_zeros))

        # check for decreasing neighbors
        candidates_right = decreasing_neighbor_check(right_shift, matr)
        candidates_left = decreasing_neighbor_check(left_shift, matr)
        candidates_top = decreasing_neighbor_check(top_shift, matr)
        candidates_down = decreasing_neighbor_check(down_shift, matr)

        # aggregate
        new_cluster_points = (candidates_right | candidates_left | candidates_top | candidates_down) & to_be_classified

        n_new_points = np.sum(new_cluster_points > 0)

        # update cluster and to_be_classified
        this_cluster[new_cluster_points] = cluster_int
        to_be_classified[new_cluster_points] = False

        if n_new_points == 0:
            
            print(f'New cluster done: {cluster_int} with {np.sum(this_cluster > 0)} points')
            cluster_not_final = False
            all_clusters[this_cluster > 0] = cluster_int

#             fig, (ax0, ax1) = plt.subplots(1, 2, figsize = (10, 10))

#             # visualize current cluster
#             ax0.imshow(this_cluster)
#             ax1.imshow(to_be_classified)
            
#             plt.show()
            
            cluster_int += 1
            


In [ ]:
all_clusters_disp = all_clusters.copy()
all_clusters_disp[all_clusters == 0] = -200

In [ ]:
# Plot raster
fig, (ax0, ax1) = plt.subplots(1, 2, figsize = (10, 10))

img_dataarr.plot.imshow(ax=ax0)
building_footprints.plot(ax=ax0, alpha=0.25, edgecolor='None', facecolor='orange')
building_footprints.plot(ax=ax0, alpha=0.75, edgecolor='red', facecolor='None')

ax1.imshow(all_clusters_disp)
plt.show()

### Transform to vector geometries

### Compute score